In [99]:
# import warnings
# warnings.simplefilter(action='ignore')

# import pandas as pd
# import numpy as np
# import datetime
# amap=pd.read_hdf('Processed_Data/amap.h5',key='1210update')
# df=pd.read_excel('DataSource\高德拥堵延时指数raw data 2023 01 16.xlsx')
# # NEWday
# # NEWday=

# for col in ['来源','频率','单位']:
#     if col in df.columns:
#         df=df.drop(columns=col)
# df=df.dropna(subset=['指标名称'])
# c1=df['指标名称'].str.contains('数据来源')
# df=df[~c1]
# df['指标名称']=df['指标名称'].str.replace('拥堵延时指数:','')
# df['城市'] = df['指标名称']
# df=df.drop(columns=['指标名称'])
# df=pd.melt(df,id_vars=['城市'],var_name='date', value_name='traffic')
# df['date']=pd.to_datetime(df['date'])
# c1=df['traffic']>0
# df=df[c1]

# updated=pd.concat([amap,df]).drop_duplicates().sort_values(by=['date','城市'])
# updated.to_hdf('Processed_Data/amap.h5',key='0116update')

,城市,date,traffic
184084,三亚,2018-01-01,1.58
184019,上海,2018-01-01,1.28
184078,东莞,2018-01-01,1.47
184099,中国香港,2018-01-01,1.25
184079,中山,2018-01-01,1.75
...,...,...,...
58,镇江,2023-01-15,1.53
19,长春,2023-01-15,1.39
12,长沙,2023-01-15,1.44
41,青岛,2023-01-15,1.36


In [101]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

amap=pd.read_hdf('Processed_Data/amap.h5',key='0116update')
NEWday=pd.read_excel('./DataSource/DailyUpdateForTG.xlsx')
NEWday=pd.melt(NEWday,id_vars=['城市'],var_name='date', value_name='traffic')
updated=pd.concat([amap,NEWday]).drop_duplicates()
updated.to_hdf('Processed_Data/amap.h5',key='0116update')

amap=updated
amap['date']=pd.to_datetime(amap['date'])
amap['WeekDay']=amap['date'].dt.strftime('%A')


def getyr(d):
    return str(d.isocalendar()[0])
def getdt(d):
    return str(d.date())[-5:]
def getMon(d):
    return str(d.date()).split('-')[1]

amap['yr']=amap['date'].apply(getyr)
amap['dt']=amap['date'].apply(getdt)
amap['mon']=amap['date'].apply(getMon)

amap=amap.sort_values(by='date',ascending=True).reset_index(drop=True)

c1=amap['date'].astype('str')>'2018-11-01'
c2=amap['date'].astype('str')<'2019-10-01'

amap_2019=amap[c1&c2]

amap_2019=amap_2019.groupby(by=['城市','mon','WeekDay'])['traffic'].mean().reset_index() # 当的traffic 变化
amap_2019=amap_2019.rename(columns={"traffic":"traffic_19_same_wd_mean"})

amap_2019_CNY=amap[c1&c2]
amap_2019_CNY['amap_last7d_mean']=amap_2019_CNY.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 7).mean())
amap_2019_CNY['amapCNY19_last7d_mean']=amap_2019_CNY.groupby('城市')['amap_last7d_mean'].transform(lambda x: x.shift(periods=-14))

amap_2019_CNY=amap_2019_CNY[['城市','dt','amapCNY19_last7d_mean']]

amap

,城市,date,traffic,WeekDay,yr,dt,mon
0,三亚,2018-01-01,1.58,Monday,2018,01-01,01
1,盐城,2018-01-01,1.41,Monday,2018,01-01,01
2,珠海,2018-01-01,1.43,Monday,2018,01-01,01
3,烟台,2018-01-01,1.40,Monday,2018,01-01,01
4,潍坊,2018-01-01,1.45,Monday,2018,01-01,01
...,...,...,...,...,...,...,...
184688,大连,2023-01-15,1.39,Sunday,2023,01-15,01
184689,大同,2023-01-15,1.59,Sunday,2023,01-15,01
184690,唐山,2023-01-15,1.41,Sunday,2023,01-15,01
184691,张家口,2023-01-15,1.38,Sunday,2023,01-15,01


In [108]:
c1=amap['date'].astype('str')>='2022-01-01'

amap_2022=amap[c1]

amap_2022=amap_2022.merge(amap_2019,how='left',on=['城市','mon','WeekDay']) # compare to calendar year
amap_2022=amap_2022.merge(amap_2019_CNY,how='left',on=['城市','dt']) #compare to CNY

amap_2022['traffic_last7d_mean'] = amap_2022.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 7).mean())
amap_2022['traffic_change']=amap_2022['traffic_last7d_mean']/amap_2022.groupby('城市')['traffic_last7d_mean'].transform(lambda x: x.shift(periods=7))-1

amap_2022['traffic_status_old']=amap_2022['traffic_last7d_mean']/amap_2022['traffic_19_same_wd_mean']
# amap_2022['traffic_status']=amap_2022['traffic']/amap_2022['amapCNY19_last7d_mean']

amap_2022['traffic_status_last7d_mean']=amap_2022['traffic_last7d_mean']/amap_2022['amapCNY19_last7d_mean']

# amap_2022['traffic_status_last7d_mean']=amap_2022.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(7, 1).mean())
amap_2022.sort_values(by='date').tail(2)

,城市,date,traffic,WeekDay,yr,dt,mon,traffic_19_same_wd_mean,amapCNY19_last7d_mean,traffic_last7d_mean,traffic_change,traffic_status_old,traffic_status_last7d_mean
38528,福州,2023-01-15,1.40,Sunday,2023,01-15,01,1.425,1.631429,1.652857,0.139901,1.159900,1.013135
38592,韶关,2023-01-15,1.35,Sunday,2023,01-15,01,1.405,1.470000,1.442857,0.058700,1.026945,0.981535


In [109]:
# amap_2022.to_hdf('Processed_Data/amap.h5',key='noCOVID')

citydate=pd.read_hdf('Processed_Data/amap.h5',key='noCOVID')
citytier=pd.read_hdf('./Processed_Data/citypopu.h5',key='citytier')
citydate=citydate[citydate['traffic_status_last7d_mean']>0.95].merge(citytier,on='城市',how='left')
citydate['yrmon']=citydate['date'].astype('str').str[:7]
# pd.pivot_table(citydate,index='CityTier',columns=['yrmon'],values='traffic_status_old',aggfunc='count').to_clipboard()
citydate['城市date']=citydate['城市']+citydate['date'].astype('str')
citydate.to_hdf('Processed_Data/amap.h5',key='noCOVIDdf')

In [94]:
# 2.0 TG 流动性影响 by 人口拆分
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

df_amap=amap_2022.merge(city_popu,on='城市',how='inner')

def catgorize2(g):
    g=g-1
    level='5 社会面流动性无影响'
    g=float(g)
    if g <=-0.25:
        level= '0 社会面基本停滞'
    elif -0.25<g<=-0.15:
        level ='1 社会面流动性显著降低'
    elif -0.15<g<=-0.05:
        level ='4 社会面流动性轻微降低'
    elif g>-0.05:
        level ='5 社会面流动性无影响'
    return level

df_amap['traffic_change_level']=df_amap['traffic_status_last7d_mean'].apply(catgorize2)

df_amap['date']=pd.to_datetime(df_amap['date'])

pv_value=pd.pivot_table(df_amap, values='常住人口', index=['date'],columns=["traffic_change_level"], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 社会面基本停滞', '1 社会面流动性显著降低', '4 社会面流动性轻微降低', '5 社会面流动性无影响']]
pv_value=pv_value.fillna(0)
pv_value.tail(22).to_clipboard(header=None)
pv_value.tail(3)

traffic_change_level,sum,0 社会面基本停滞,1 社会面流动性显著降低,4 社会面流动性轻微降低,5 社会面流动性无影响
date,,,,,
2023-01-13,1,0.0,0.010453,0.183775,0.805772
2023-01-14,1,0.0,0.010453,0.151793,0.837755
2023-01-15,1,0.0,0.010453,0.077000,0.912547


In [3]:
# # WoW vs. COVID
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

# df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
# dfo=amap_2022.merge(df_covid,on=['城市'],how='left')
# dfo['date']=dfo['date'].astype('string') 
# dfo=dfo[dfo['date']==riqi].sort_values(by='常住人口',ascending=False).head(70)



 
# o=dfo.copy()
# o['size']=np.nan
# o['group']='其他'
# c1=o['城市'].isin(dafeng_city)
# o.loc[c1,'group']='本波疫情基本结束'
# o=o.sort_values(by='group',ascending=False)

# show=o[['城市','progress_2','traffic_change','size','group']]
# show.to_clipboard(index=False,header=None)

# print(riqi)
# show.head(3)

In [4]:
# # as of 2019 vs. COVID

# show=o[['城市','progress_2','traffic_status_last7d_mean','size','group']] #,'progress_2'
# show.to_clipboard(index=False,header=None)

# print(riqi)
# show.head(3)


In [42]:
# #set baseline
# d=amap_2022.copy()
# c1=d.date.astype('str')<='2022-11-28'
# c2=d.date.astype('str')>='2022-11-15'
# d_base=d[c1&c2].groupby(by='城市').mean().reset_index().rename(columns={'traffic':'index_base'})[['城市', 'index_base']]
# d=d.merge(d_base,on=['城市'],how='left')

# d['asof2211']=d['traffic']/d['index_base']

# amap_2022=d.copy()

In [46]:
# WoW vs. GDP

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi

riqi='2023-01-15'

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
dfo=amap_2022.merge(city_popu,on=['城市'],how='left')

d=dfo[dfo['date']==riqi][['城市','date','traffic_status_last7d_mean']].reset_index(drop=True).rename(columns={'traffic_status_last7d_mean':'amap'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='amap')

dfo['date']=dfo['date'].astype('string') 
o=dfo[dfo['date']==riqi].sort_values(by='常住人口',ascending=False).head(70)

o=o[['城市','GDP','traffic_change']] #,'progress_2'
o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by=['group','GDP'],ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)
o.head(2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-01-15


,城市,GDP,traffic_change,size,group
37718,上海,38701.0,0.107430,NaN,本波疫情基本结束
37713,北京,36103.0,0.064081,NaN,本波疫情基本结束


In [47]:
# as of 2019 vs. GDP
o=dfo[dfo['date']==riqi].sort_values(by='常住人口',ascending=False).head(100)

o=o[['城市','GDP','traffic_status_last7d_mean']] #,'progress_2'
o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by=['group','GDP'],ascending=False)
o.to_clipboard(index=False,header=None)
print(riqi)
o.head(2)

2023-01-15


,城市,GDP,traffic_status_last7d_mean,size,group
37718,上海,38701.0,1.036711,NaN,本波疫情基本结束
37713,北京,36103.0,1.021025,NaN,本波疫情基本结束


In [111]:
d1=pd.read_hdf('Processed_Data/tablehead.h5','amap')
d2= pd.pivot_table(amap[amap['date'].astype('str')>='2023-01-11'],index='城市',columns='date',values='traffic',aggfunc='mean').reset_index()
d1.merge(d2,on='城市',how='left').drop(columns='城市').to_clipboard(index=False)

In [95]:
citytier=pd.read_hdf('./Processed_Data/citypopu.h5',key='citytier')

bytier=amap_2022.merge(citytier,on='城市',how='left')
bytier['CityTier']=bytier['CityTier'].str.replace('四线城市','三线城市')
bytier=bytier.groupby(['CityTier','date']).mean().reset_index()[['CityTier','date','traffic_status_last7d_mean']]
bytier=pd.pivot_table(bytier,index='date',columns='CityTier',values='traffic_status_last7d_mean',aggfunc='mean').reset_index()

alldata=amap_2022.merge(citytier,on='城市',how='left')
alldata=alldata[~alldata['CityTier'].isnull()].groupby(['date']).mean().reset_index()[['date','traffic_status_last7d_mean']]
alldata=alldata[['date','traffic_status_last7d_mean']].rename(columns={'traffic_status_last7d_mean':'all'})

bytier.merge(alldata,how='left',on='date').tail(80).to_clipboard(index=False)
